<a href="https://colab.research.google.com/github/mostafa-asefy/neural-networks-from-zero/blob/main/Artificial_Neural_Networks_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Network Class Definition

In [ ]:
import numpy as np
from scipy.special import expit


class NeuralNetwork:

    # راه اندازی شبکه عصبی 
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        # تعیین تعداد نورون‌های لایه‌های مختلف
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes

        # تعیین نرخ یادگیری
        self.learning_rate = learning_rate
        
        # ایجاد تابع فعال سازی که اینجا سیگموییدی است ولی می تواند متفاوت باشد
        self.activation_function = lambda x : expit(x)

        # ایجاد دو ماتریس وزن‌های اولیه
        # یکی برای وزن‌های بین لایه ورودی و مخفی و یکی برای وزن‌های بین لایه مخفی و خروجی 
        # هر وزن با ایندکس آی و جی، وزن اتصال بین نورون آی از لایه مبدا به نورون جی در لایه مقصد است.
        #  و به همین ترتیب w12 w21 مثلا
        self.w_i_h = np.random.default_rng().normal(0, pow(self.input_nodes, -0.5),
                                                    (self.hidden_nodes, self.input_nodes))
        self.w_h_o = np.random.default_rng().normal(0, pow(self.hidden_nodes, -0.5),
                                                    (self.output_nodes, self.hidden_nodes))
        pass

    # یادگیری شبکه بر اساس یک نمونه ورودی/خروجی - پس انتشار
    def train():
        pass

    # محاسبه خروجی شبکه با دادن ورودی - پیش خور
    def query(self, input_list):
        # تبدیل لیست مقادیر ورودی به آرایه دوبعدی
        inputs = np.array(input_list, ndmin=2).T

        # محاسبه سیگنال ورودی و سپس خروجی لایه مخفی
        x_hidden = np.dot(self.w_i_h, inputs)
        o_hidden = self.activation_function(x_hidden)

        # محاسبه سیگنال ورودی و سپس خروجی لایه خروجی
        x_output = np.dot(self.w_h_o, o_hidden)
        o_output = self.activation_function(x_output)

        return o_output


# Neural Network Instance Creation

In [ ]:
nn = NeuralNetwork(3, 3, 3, 0.3)
print(nn.w_i_h)
nn.query([1.0, 0.5, -1.5])

[[-0.62818949 -0.38082066  0.3034825 ]
 [-0.02442757  0.26714613  0.34320552]
 [ 0.16222883  0.23624561 -0.74265773]]


array([[0.2410514 ],
       [0.55568368],
       [0.47806991]])